In [1]:
from library_dicom.dicom_processor.model.SeriesPT import SeriesPT
from library_dicom.dicom_processor.model.SeriesCT import SeriesCT

from library_dicom.dicom_processor.model.FusionPET_CT import FusionPET_CT

import SimpleITK as sitk 
import tensorflow as tf 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
csv_path = '/media/deeplearning/Elements/AHL2011/AHL_NIFTI/AHL2011_NIFTI_V4.csv'

In [3]:
df = pd.read_csv(csv_path)
df = df[df['PET'] == 'pet0'] 
df.head()

key_split = 'PATIENT_ID'  # unique id
idx = np.arange(df[key_split].nunique()) #0 to number of PET0
split = np.empty(df[key_split].nunique(), dtype="<U6")

idx_train, idx_test = train_test_split(idx, test_size=0.15)
idx_train, idx_val = train_test_split(idx_train, test_size=0.15)

In [4]:
split[idx_train] = 'train'
split[idx_val] = 'val'
split[idx_test] = 'test'

In [5]:
split

array(['train', 'train', 'train', 'val', 'test', 'train', 'train',
       'train', 'train', 'train', 'train', 'train', 'train', 'train',
       'test', 'train', 'train', 'train', 'test', 'train', 'val', 'train',
       'train', 'train', 'train', 'train', 'train', 'train', 'train',
       'train', 'train', 'train', 'val', 'train', 'train', 'val', 'train',
       'train', 'train', 'train', 'train', 'train', 'val', 'train',
       'train', 'val', 'test', 'train', 'train', 'val', 'val', 'train',
       'train', 'train', 'train', 'train', 'train', 'train', 'train',
       'train', 'train', 'train', 'train', 'train', 'train', 'test',
       'train', 'train', 'train', 'train', 'train', 'train', 'train',
       'train', 'train', 'train', 'train', 'train', 'train', 'train',
       'train', 'train', 'train', 'train', 'train', 'train', 'train',
       'train', 'train', 'test', 'train', 'test', 'val', 'train', 'train',
       'train', 'train', 'train', 'train', 'train', 'train', 'train',
       'v

In [6]:
df_patient = pd.DataFrame(data={key_split: df[key_split].unique(),
                                                'subset': split})
df = df.merge(df_patient, on=key_split, how='left')
df_train = df[df['subset'] == 'train']
df_val = df[df['subset'] == 'val']
df_test = df[df['subset'] == 'test']
df.head()

,PATIENT_NAME,PATIENT_ID,STUDY_UID,PET,NIFTI_CT,NIFTI_PET,NIFTI_MASK,subset
0,"CON,DA",13011101331015,1.2.124.113532.10.149.172.5.20131107.93455.772...,pet0,AHL_NIFTI/13011101331015/pet0/1.2.124.113532.1...,AHL_NIFTI/13011101331015/pet0/1.2.124.113532.1...,AHL_NIFTI/13011101331015/pet0/1.2.124.113532.1...,train
1,"BOU,SA",13011101051001,1.2.124.113532.172.16.25.2.20110720.111654.943617,pet0,AHL_NIFTI/13011101051001/pet0/1.2.124.113532.1...,AHL_NIFTI/13011101051001/pet0/1.2.124.113532.1...,AHL_NIFTI/13011101051001/pet0/1.2.124.113532.1...,train
2,"BOU,YA",13011101431006,1.2.124.113532.172.16.25.2.20120125.456.1038547,pet0,AHL_NIFTI/13011101431006/pet0/1.2.124.113532.1...,AHL_NIFTI/13011101431006/pet0/1.2.124.113532.1...,AHL_NIFTI/13011101431006/pet0/1.2.124.113532.1...,train
3,"SAN,MA",13011101431010,1.2.124.113532.172.16.25.2.20120605.739.1104125,pet0,AHL_NIFTI/13011101431010/pet0/1.2.124.113532.1...,AHL_NIFTI/13011101431010/pet0/1.2.124.113532.1...,AHL_NIFTI/13011101431010/pet0/1.2.124.113532.1...,val
4,"RIG,EL",13011101431009,1.2.124.113532.172.16.25.2.20120606.612.1104622,pet0,AHL_NIFTI/13011101431009/pet0/1.2.124.113532.1...,AHL_NIFTI/13011101431009/pet0/1.2.124.113532.1...,AHL_NIFTI/13011101431009/pet0/1.2.124.113532.1...,test


In [7]:
def wrap_in_list_of_dict(df):
        """
        :return: [ {'pet_img': pet_img0_path, 'ct_img': ct_img0_path, 'mask_img': mask_img0_path, 'merged_img': merged_img0},
                    {'pet_img': pet_img1_path, 'ct_img': ct_img1_path, 'mask_img': mask_img1_path,  'merged_img': merged_img1},
                    {'pet_img': pet_img2_path, 'ct_img': ct_img2_path, 'mask_img': mask_img2_path,  'merged_img': merged_img2}, ...]
        """
        # return df[['NIFTI_PET', 'NIFTI_CT', 'NIFTI_MASK']].T.to_dict().values()
        mapper = {'NIFTI_PET': 'pet_img', 'NIFTI_CT': 'ct_img', 'NIFTI_MASK': 'mask_img'}
        return df[['NIFTI_PET', 'NIFTI_CT', 'NIFTI_MASK']].rename(columns=mapper).to_dict('records')

In [10]:
df = wrap_in_list_of_dict(df[df['subset'] == 'train'])

In [15]:
df[0].items()

dict_items([('pet_img', 'AHL_NIFTI/13011101331015/pet0/1.2.124.113532.10.149.172.5.20131107.93455.7725981_nifti_PT.nii'), ('ct_img', 'AHL_NIFTI/13011101331015/pet0/1.2.124.113532.10.149.172.5.20131107.93455.7725981_nifti_CT.nii'), ('mask_img', 'AHL_NIFTI/13011101331015/pet0/1.2.124.113532.10.149.172.5.20131107.93455.7725981_nifti_mask.nii')])